In [2]:
import requests
from urllib.request import Request, urlopen
from urllib.parse import urlencode, quote_plus
from bs4 import BeautifulSoup

import numpy as np
import pandas as pd
import csv
import time
import json
from pandas import json_normalize

In [5]:
data = pd.read_csv("임대주택단지.csv", encoding='cp949')
display(data)

,Unnamed: 0,전용면적,월임대료,총세대수,데이터순번,단지명,최초입주년월,지역명,세대수,전체건수,공급유형명,임대보증금
0,0,46.7100,358790,873,1,서울강남 3블록,201311.0,서울특별시 강남구,72,19,국민임대,43231000
1,1,46.0500,358790,873,2,서울강남 3블록,201311.0,서울특별시 강남구,64,19,국민임대,43231000
2,2,46.0100,358790,873,3,서울강남 3블록,201311.0,서울특별시 강남구,180,19,국민임대,43231000
3,3,36.6200,252360,873,4,서울강남 3블록,201311.0,서울특별시 강남구,84,19,국민임대,20188000
4,4,36.0800,252360,873,5,서울강남 3블록,201311.0,서울특별시 강남구,448,19,국민임대,20188000
...,...,...,...,...,...,...,...,...,...,...,...,...
2981,2981,46.7203,81540,151,66,함덕진우,NaN,제주특별자치도 제주시,16,70,국민임대,14833000
2982,2982,37.2671,50350,151,67,함덕진우,NaN,제주특별자치도 제주시,3,70,국민임대,9221000
2983,2983,37.2638,50350,151,68,함덕진우,NaN,제주특별자치도 제주시,3,70,국민임대,9221000
2984,2984,37.2011,55150,151,69,함덕진우,NaN,제주특별자치도 제주시,4,70,국민임대,10036000


In [6]:
data.shape, data.drop_duplicates().shape # 중복 체크

((2986, 12), (2986, 12))

In [11]:
data = data.rename(columns={'지역명':'지역'})
display(data)

df_apt = data['지역'] == '서울특별시 강남구'
df1 = data[df_apt]
display(df1)

# df_apt1 = df2['임대건물구분'] == '아파트'
# df2 = df2[df_apt1]
# display(df2)

,Unnamed: 0,전용면적,월임대료,총세대수,데이터순번,단지명,최초입주년월,지역,세대수,전체건수,공급유형명,임대보증금
0,0,46.7100,358790,873,1,서울강남 3블록,201311.0,서울특별시 강남구,72,19,국민임대,43231000
1,1,46.0500,358790,873,2,서울강남 3블록,201311.0,서울특별시 강남구,64,19,국민임대,43231000
2,2,46.0100,358790,873,3,서울강남 3블록,201311.0,서울특별시 강남구,180,19,국민임대,43231000
3,3,36.6200,252360,873,4,서울강남 3블록,201311.0,서울특별시 강남구,84,19,국민임대,20188000
4,4,36.0800,252360,873,5,서울강남 3블록,201311.0,서울특별시 강남구,448,19,국민임대,20188000
...,...,...,...,...,...,...,...,...,...,...,...,...
2981,2981,46.7203,81540,151,66,함덕진우,NaN,제주특별자치도 제주시,16,70,국민임대,14833000
2982,2982,37.2671,50350,151,67,함덕진우,NaN,제주특별자치도 제주시,3,70,국민임대,9221000
2983,2983,37.2638,50350,151,68,함덕진우,NaN,제주특별자치도 제주시,3,70,국민임대,9221000
2984,2984,37.2011,55150,151,69,함덕진우,NaN,제주특별자치도 제주시,4,70,국민임대,10036000


,Unnamed: 0,전용면적,월임대료,총세대수,데이터순번,단지명,최초입주년월,지역,세대수,전체건수,공급유형명,임대보증금
0,0,46.71,358790,873,1,서울강남 3블록,201311.0,서울특별시 강남구,72,19,국민임대,43231000
1,1,46.05,358790,873,2,서울강남 3블록,201311.0,서울특별시 강남구,64,19,국민임대,43231000
2,2,46.01,358790,873,3,서울강남 3블록,201311.0,서울특별시 강남구,180,19,국민임대,43231000
3,3,36.62,252360,873,4,서울강남 3블록,201311.0,서울특별시 강남구,84,19,국민임대,20188000
4,4,36.08,252360,873,5,서울강남 3블록,201311.0,서울특별시 강남구,448,19,국민임대,20188000
5,5,36.05,252360,873,6,서울강남 3블록,201311.0,서울특별시 강남구,25,19,국민임대,20188000


In [4]:
강원 = pd.read_csv("공공임대추출데이터/공공임대주택단지_강원_아파트.csv", encoding='cp949')
경남 = pd.read_csv("공공임대추출데이터/공공임대주택단지_경남_아파트.csv", encoding='cp949')
울산 = pd.read_csv("공공임대추출데이터/공공임대주택단지_울산_아파트.csv", encoding='cp949')
display(강원)
display(경남)
display(울산)

강원 = 강원.rename(columns={'공급전용면적':'전용면적', '광역시도 명':'지역'})
경남 = 경남.rename(columns={'공급전용면적':'전용면적', '광역시도 명':'지역'})
울산 = 울산.rename(columns={'공급전용면적':'전용면적', '광역시도 명':'지역'})

display(강원)
display(경남)
display(울산)

,Unnamed: 0,전체결과수,단지 식별자,기관 명,광역시도 코드,광역시도 명,시군구 코드,지역구,시군구,주소,세대수,공급유형,형 명,공급전용면적,공급공용면적,주택유형,주차수,임대보증금,월임대료,준공일자
0,358,738,31120083,LH강원,42,강원도,110,춘천시,춘천거두2 행복주택,강원도 춘천시 동내면 거두택지길 70,480,행복주택,16,16.910,10.4397,아파트,0,15520000,73720,20180223.0
1,737,738,3653,강원도개발공사,42,강원도,110,춘천시,춘천신촌산수빌아파트,강원도 춘천시 동내면 향군길 23,163,국민임대,22,51.360,34.8300,아파트,212,22687000,112500,20050729.0
2,843,18,30699446,LH강원,42,강원도,230,삼척시,삼척도계2단지,강원도 삼척시 도계읍 장원길 33,280,10년임대,59,59.750,24.8741,아파트,230,23687000,276340,20131130.0
3,844,18,30699446,LH강원,42,강원도,230,삼척시,삼척도계2단지,강원도 삼척시 도계읍 장원길 33,280,10년임대,59,59.750,25.0741,아파트,230,23687000,276340,20131130.0
4,845,18,30699446,LH강원,42,강원도,230,삼척시,삼척도계2단지,강원도 삼척시 도계읍 장원길 33,280,10년임대,59,59.750,25.2641,아파트,230,23687000,276340,20131130.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
140,739,95,31371459,LH강원,42,강원도,170,동해시,강원도 동해시,강원도 동해시 평릉1길 37-1,7,매입임대,49,49.950,15.0228,아파트,0,0,0,NaN
141,740,95,31371459,LH강원,42,강원도,170,동해시,강원도 동해시,강원도 동해시 평릉1길 37-1,7,매입임대,60,60.000,12.5533,아파트,0,0,0,NaN
142,741,95,31371459,LH강원,42,강원도,170,동해시,강원도 동해시,강원도 동해시 평릉1길 37-1,7,매입임대,70,70.435,22.6660,아파트,0,0,0,NaN
143,742,95,31371459,LH강원,42,강원도,170,동해시,강원도 동해시,강원도 동해시 평릉1길 37-1,7,매입임대,71,71.839,22.2300,아파트,0,0,0,NaN


,Unnamed: 0,전체결과수,단지 식별자,기관 명,광역시도 코드,광역시도 명,시군구 코드,지역구,시군구,주소,세대수,공급유형,형 명,공급전용면적,공급공용면적,주택유형,주차수,임대보증금,월임대료,준공일자
0,351,1353,31186807,LH경남,48,경상남도,170,진주시,진주문산 LH아파트,경상남도 진주시 문산읍 월아산로 1047-14,210,행복주택,16,16.7100,8.9603,아파트,112,13880000,62220,20180825.0
1,356,1353,31148737,LH경남,48,경상남도,170,진주시,혁신도시 LH아파트 3단지,경상남도 진주시 대밭골로 91,966,행복주택,21,21.5400,14.2110,아파트,879,20460000,100000,20180430.0
2,357,1353,31148737,LH경남,48,경상남도,170,진주시,혁신도시 LH아파트 3단지,경상남도 진주시 대밭골로 91,966,행복주택,21,21.9000,14.4485,아파트,879,21220000,104000,20180430.0
3,354,1353,31148737,LH경남,48,경상남도,170,진주시,혁신도시 LH아파트 3단지,경상남도 진주시 대밭골로 91,80,영구임대,24,24.7400,16.3222,아파트,879,12117000,108000,20180430.0
4,358,1353,31148737,LH경남,48,경상남도,170,진주시,혁신도시 LH아파트 3단지,경상남도 진주시 대밭골로 91,966,행복주택,26,26.2700,17.3316,아파트,879,24660000,121000,20180430.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
122,221,1353,31261216,LH경남,48,경상남도,170,진주시,경상남도 진주시,경상남도 진주시 금산면 중장로154번길 49,1,매입임대,84,84.9950,26.1980,아파트,0,80640000,222320,NaN
123,222,1353,31261215,LH경남,48,경상남도,170,진주시,경상남도 진주시,경상남도 진주시 금산면 중장로154번길 49,1,매입임대,84,84.9950,26.1980,아파트,0,82560000,227680,NaN
124,239,1353,31259923,LH경남,48,경상남도,170,진주시,매입다가구(경남진주시),경상남도 진주시 금산면 중장로154번길 49,1,매입임대,84,84.9950,26.1980,아파트,0,35000000,219620,NaN
125,153,1353,31269086,LH경남,48,경상남도,170,진주시,경상남도 진주시,경상남도 진주시 금산면 금산로 22,1,매입임대,84,84.9968,25.5140,아파트,0,76800000,206920,NaN


,Unnamed: 0,전체결과수,단지 식별자,기관 명,광역시도 코드,광역시도 명,시군구 코드,지역구,시군구,주소,세대수,공급유형,형 명,공급전용면적,공급공용면적,주택유형,주차수,임대보증금,월임대료,준공일자
0,436,648,30819489,LH부산울산,31,울산광역시,110,중구,울산우정3단지,울산광역시 중구 유곡로 99,66,10년임대,59,59.96,22.2873,아파트,0,40690000,574030,20140905
1,437,648,30819489,LH부산울산,31,울산광역시,110,중구,울산우정3단지,울산광역시 중구 유곡로 99,66,10년임대,59,59.97,22.2910,아파트,0,41021000,576600,20140905
2,642,648,30700425,LH부산울산,31,울산광역시,110,중구,울산 우정1단지,울산광역시 중구 종가로 668-9,736,국민임대,36,36.83,15.7051,아파트,0,15113000,233470,20130313
3,643,648,30700425,LH부산울산,31,울산광역시,110,중구,울산 우정1단지,울산광역시 중구 종가로 668-9,736,국민임대,46,46.69,19.9097,아파트,0,32823000,309050,20130313
4,644,648,30700425,LH부산울산,31,울산광역시,110,중구,울산 우정1단지,울산광역시 중구 종가로 668-9,736,국민임대,51,51.60,22.0034,아파트,0,36883000,360950,20130313
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
84,2544,310,30699568,LH부산울산,31,울산광역시,710,울주군,울산구영 주공2단지,울산광역시 울주군 범서읍 점촌3길 23,407,국민임대,46,46.90,20.8042,아파트,459,19895000,160400,20070924
85,2545,310,30699560,LH부산울산,31,울산광역시,710,울주군,울산구영 주공1단지,울산광역시 울주군 범서읍 대리로 43,708,국민임대,51,51.79,19.1663,아파트,711,22471000,177880,20070418
86,2546,310,30699560,LH부산울산,31,울산광역시,710,울주군,울산구영 주공1단지,울산광역시 울주군 범서읍 대리로 43,708,국민임대,51,51.84,19.1848,아파트,711,22471000,177880,20070418
87,2547,310,30699560,LH부산울산,31,울산광역시,710,울주군,울산구영 주공1단지,울산광역시 울주군 범서읍 대리로 43,708,국민임대,51,51.92,19.2144,아파트,711,22471000,177880,20070418


,Unnamed: 0,전체결과수,단지 식별자,기관 명,광역시도 코드,지역,시군구 코드,지역구,시군구,주소,세대수,공급유형,형 명,전용면적,공급공용면적,주택유형,주차수,임대보증금,월임대료,준공일자
0,358,738,31120083,LH강원,42,강원도,110,춘천시,춘천거두2 행복주택,강원도 춘천시 동내면 거두택지길 70,480,행복주택,16,16.910,10.4397,아파트,0,15520000,73720,20180223.0
1,737,738,3653,강원도개발공사,42,강원도,110,춘천시,춘천신촌산수빌아파트,강원도 춘천시 동내면 향군길 23,163,국민임대,22,51.360,34.8300,아파트,212,22687000,112500,20050729.0
2,843,18,30699446,LH강원,42,강원도,230,삼척시,삼척도계2단지,강원도 삼척시 도계읍 장원길 33,280,10년임대,59,59.750,24.8741,아파트,230,23687000,276340,20131130.0
3,844,18,30699446,LH강원,42,강원도,230,삼척시,삼척도계2단지,강원도 삼척시 도계읍 장원길 33,280,10년임대,59,59.750,25.0741,아파트,230,23687000,276340,20131130.0
4,845,18,30699446,LH강원,42,강원도,230,삼척시,삼척도계2단지,강원도 삼척시 도계읍 장원길 33,280,10년임대,59,59.750,25.2641,아파트,230,23687000,276340,20131130.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
140,739,95,31371459,LH강원,42,강원도,170,동해시,강원도 동해시,강원도 동해시 평릉1길 37-1,7,매입임대,49,49.950,15.0228,아파트,0,0,0,NaN
141,740,95,31371459,LH강원,42,강원도,170,동해시,강원도 동해시,강원도 동해시 평릉1길 37-1,7,매입임대,60,60.000,12.5533,아파트,0,0,0,NaN
142,741,95,31371459,LH강원,42,강원도,170,동해시,강원도 동해시,강원도 동해시 평릉1길 37-1,7,매입임대,70,70.435,22.6660,아파트,0,0,0,NaN
143,742,95,31371459,LH강원,42,강원도,170,동해시,강원도 동해시,강원도 동해시 평릉1길 37-1,7,매입임대,71,71.839,22.2300,아파트,0,0,0,NaN


,Unnamed: 0,전체결과수,단지 식별자,기관 명,광역시도 코드,지역,시군구 코드,지역구,시군구,주소,세대수,공급유형,형 명,전용면적,공급공용면적,주택유형,주차수,임대보증금,월임대료,준공일자
0,351,1353,31186807,LH경남,48,경상남도,170,진주시,진주문산 LH아파트,경상남도 진주시 문산읍 월아산로 1047-14,210,행복주택,16,16.7100,8.9603,아파트,112,13880000,62220,20180825.0
1,356,1353,31148737,LH경남,48,경상남도,170,진주시,혁신도시 LH아파트 3단지,경상남도 진주시 대밭골로 91,966,행복주택,21,21.5400,14.2110,아파트,879,20460000,100000,20180430.0
2,357,1353,31148737,LH경남,48,경상남도,170,진주시,혁신도시 LH아파트 3단지,경상남도 진주시 대밭골로 91,966,행복주택,21,21.9000,14.4485,아파트,879,21220000,104000,20180430.0
3,354,1353,31148737,LH경남,48,경상남도,170,진주시,혁신도시 LH아파트 3단지,경상남도 진주시 대밭골로 91,80,영구임대,24,24.7400,16.3222,아파트,879,12117000,108000,20180430.0
4,358,1353,31148737,LH경남,48,경상남도,170,진주시,혁신도시 LH아파트 3단지,경상남도 진주시 대밭골로 91,966,행복주택,26,26.2700,17.3316,아파트,879,24660000,121000,20180430.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
122,221,1353,31261216,LH경남,48,경상남도,170,진주시,경상남도 진주시,경상남도 진주시 금산면 중장로154번길 49,1,매입임대,84,84.9950,26.1980,아파트,0,80640000,222320,NaN
123,222,1353,31261215,LH경남,48,경상남도,170,진주시,경상남도 진주시,경상남도 진주시 금산면 중장로154번길 49,1,매입임대,84,84.9950,26.1980,아파트,0,82560000,227680,NaN
124,239,1353,31259923,LH경남,48,경상남도,170,진주시,매입다가구(경남진주시),경상남도 진주시 금산면 중장로154번길 49,1,매입임대,84,84.9950,26.1980,아파트,0,35000000,219620,NaN
125,153,1353,31269086,LH경남,48,경상남도,170,진주시,경상남도 진주시,경상남도 진주시 금산면 금산로 22,1,매입임대,84,84.9968,25.5140,아파트,0,76800000,206920,NaN


,Unnamed: 0,전체결과수,단지 식별자,기관 명,광역시도 코드,지역,시군구 코드,지역구,시군구,주소,세대수,공급유형,형 명,전용면적,공급공용면적,주택유형,주차수,임대보증금,월임대료,준공일자
0,436,648,30819489,LH부산울산,31,울산광역시,110,중구,울산우정3단지,울산광역시 중구 유곡로 99,66,10년임대,59,59.96,22.2873,아파트,0,40690000,574030,20140905
1,437,648,30819489,LH부산울산,31,울산광역시,110,중구,울산우정3단지,울산광역시 중구 유곡로 99,66,10년임대,59,59.97,22.2910,아파트,0,41021000,576600,20140905
2,642,648,30700425,LH부산울산,31,울산광역시,110,중구,울산 우정1단지,울산광역시 중구 종가로 668-9,736,국민임대,36,36.83,15.7051,아파트,0,15113000,233470,20130313
3,643,648,30700425,LH부산울산,31,울산광역시,110,중구,울산 우정1단지,울산광역시 중구 종가로 668-9,736,국민임대,46,46.69,19.9097,아파트,0,32823000,309050,20130313
4,644,648,30700425,LH부산울산,31,울산광역시,110,중구,울산 우정1단지,울산광역시 중구 종가로 668-9,736,국민임대,51,51.60,22.0034,아파트,0,36883000,360950,20130313
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
84,2544,310,30699568,LH부산울산,31,울산광역시,710,울주군,울산구영 주공2단지,울산광역시 울주군 범서읍 점촌3길 23,407,국민임대,46,46.90,20.8042,아파트,459,19895000,160400,20070924
85,2545,310,30699560,LH부산울산,31,울산광역시,710,울주군,울산구영 주공1단지,울산광역시 울주군 범서읍 대리로 43,708,국민임대,51,51.79,19.1663,아파트,711,22471000,177880,20070418
86,2546,310,30699560,LH부산울산,31,울산광역시,710,울주군,울산구영 주공1단지,울산광역시 울주군 범서읍 대리로 43,708,국민임대,51,51.84,19.1848,아파트,711,22471000,177880,20070418
87,2547,310,30699560,LH부산울산,31,울산광역시,710,울주군,울산구영 주공1단지,울산광역시 울주군 범서읍 대리로 43,708,국민임대,51,51.92,19.2144,아파트,711,22471000,177880,20070418


In [14]:
# 결측값 : 경남 강원
경남지역 = df1['지역'] == '경상남도'
df1경남 = df1[경남지역]

강원지역 = df1['지역'] == '강원도'
df1강원 = df1[강원지역]

display(df1경남)
display(df1강원)

merge_df1경남 = pd.merge(df1경남, 경남, on='전용면적')
merge_df1경남.to_csv('지역별합친데이터/트레인_경남_아파트_merge_cp949.csv', encoding='cp949')


merge_df1강원 = pd.merge(df1강원, 강원, on='전용면적')
merge_df1강원.to_csv('지역별합친데이터/트레인_강원_아파트_merge_cp949.csv', encoding='cp949')

,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,도보 10분거리 내 지하철역 수(환승노선 수 반영),도보 10분거리 내 버스정류장 수,단지내주차면수,등록차량수
8,C2515,545,아파트,경상남도,국민임대,33.48,276,17.0,A,9216000,82940,0.0,3.0,624.0,205.0
9,C2515,545,아파트,경상남도,국민임대,39.60,60,17.0,A,12672000,107130,0.0,3.0,624.0,205.0
10,C2515,545,아파트,경상남도,국민임대,39.60,20,17.0,A,12672000,107130,0.0,3.0,624.0,205.0
11,C2515,545,아파트,경상남도,국민임대,46.90,38,17.0,A,18433000,149760,0.0,3.0,624.0,205.0
12,C2515,545,아파트,경상남도,국민임대,46.90,19,17.0,A,18433000,149760,0.0,3.0,624.0,205.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2844,C2225,1206,아파트,경상남도,행복주택,26.17,120,49.0,K,51961000,27060,0.0,3.0,842.0,448.0
2845,C2225,1206,아파트,경상남도,행복주택,26.19,16,49.0,K,51961000,27060,0.0,3.0,842.0,448.0
2846,C2225,1206,아파트,경상남도,행복주택,36.03,290,49.0,K,72294000,37650,0.0,3.0,842.0,448.0
2847,C2225,1206,아파트,경상남도,행복주택,36.03,60,49.0,K,72294000,37650,0.0,3.0,842.0,448.0


,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,도보 10분거리 내 지하철역 수(환승노선 수 반영),도보 10분거리 내 버스정류장 수,단지내주차면수,등록차량수
67,C2073,361,아파트,강원도,국민임대,39.63,94,13.0,A,10813000,144180,0.0,5.0,288.0,301.0
68,C2073,361,아파트,강원도,국민임대,39.63,74,13.0,A,10813000,144180,0.0,5.0,288.0,301.0
69,C2073,361,아파트,강원도,국민임대,46.90,97,13.0,A,16821000,205460,0.0,5.0,288.0,301.0
70,C2073,361,아파트,강원도,국민임대,46.90,46,13.0,A,16821000,205460,0.0,5.0,288.0,301.0
71,C2073,361,아파트,강원도,국민임대,46.96,10,13.0,A,16821000,205460,0.0,5.0,288.0,301.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2947,C2532,239,아파트,강원도,국민임대,49.20,19,7.0,A,11346000,116090,0.0,1.0,166.0,146.0
2948,C2532,239,아파트,강원도,국민임대,51.08,34,7.0,A,14005000,142310,0.0,1.0,166.0,146.0
2949,C2532,239,아파트,강원도,국민임대,51.73,34,7.0,A,14005000,142310,0.0,1.0,166.0,146.0
2950,C2532,239,아파트,강원도,국민임대,51.96,114,7.0,A,14005000,142310,0.0,1.0,166.0,146.0


In [16]:
# 결측값 : 강원 울산 경남
강원지역 = df2['지역'] == '강원도'
df2강원 = df2[강원지역]

울산지역 = df2['지역'] == '울산광역시'
df2울산 = df2[울산지역]

경남지역 = df2['지역'] == '경상남도'
df2경남 = df2[경남지역]

merge_df2강원 = pd.merge(df2강원, 강원, on='전용면적')
merge_df2강원.to_csv('지역별합친데이터/테스트_강원_아파트_merge_cp949.csv', encoding='cp949')

merge_df2울산 = pd.merge(df2울산, 울산, on='전용면적')
merge_df2울산.to_csv('지역별합친데이터/테스트_울산_아파트_merge_cp949.csv', encoding='cp949')

merge_df2경남 = pd.merge(df2경남, 경남, on='전용면적')
merge_df2경남.to_csv('지역별합친데이터/테스트_경남_아파트_merge_cp949.csv', encoding='cp949')

display(merge_df2강원)
display(merge_df2울산)
display(merge_df2경남)

,단지코드,총세대수,임대건물구분,지역_x,공급유형_x,전용면적,전용면적별세대수,공가수,자격유형,임대보증금_x,...,주소,세대수,공급유형_y,형 명,공급공용면적,주택유형,주차수,임대보증금_y,월임대료,준공일자
0,C2523,371,아파트,강원도,국민임대,36.65,98,2.0,A,14943000,...,강원도 춘천시 동내면 거두택지길 88,375,국민임대,36,16.7212,아파트,360,11983000,102350,20090331.0
1,C2523,371,아파트,강원도,국민임대,46.90,208,2.0,A,21442000,...,강원도 삼척시 엑스포로 174,503,국민임대,46,18.1971,아파트,375,18726000,112340,20061220.0
2,C2523,371,아파트,강원도,국민임대,46.90,208,2.0,A,21442000,...,강원도 삼척시 도계읍 도계로 134,225,국민임대,46,17.6125,아파트,186,22329000,84350,20060614.0
3,C2523,371,아파트,강원도,국민임대,46.90,208,2.0,A,21442000,...,강원도 춘천시 동면 후석로326번길 96,747,국민임대,46,20.6309,아파트,698,22471000,168520,20091004.0
4,C1772,496,아파트,강원도,국민임대,46.90,306,25.0,A,18101000,...,강원도 삼척시 엑스포로 174,503,국민임대,46,18.1971,아파트,375,18726000,112340,20061220.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
104,C1289,482,아파트,강원도,국민임대,26.91,124,15.0,H,9494000,...,강원도 춘천시 우두로 130,482,국민임대,26,13.7990,아파트,0,9494000,109220,20171201.0
105,C1289,482,아파트,강원도,국민임대,26.97,100,15.0,H,9494000,...,강원도 춘천시 우두로 130,482,국민임대,26,13.8298,아파트,0,14634000,109220,20171201.0
106,C1289,482,아파트,강원도,국민임대,36.96,124,15.0,H,13041000,...,강원도 춘천시 우두로 130,482,국민임대,36,18.9525,아파트,0,13041000,181400,20171201.0
107,C1289,482,아파트,강원도,국민임대,36.96,72,15.0,H,13041000,...,강원도 춘천시 우두로 130,482,국민임대,36,18.9525,아파트,0,13041000,181400,20171201.0


,단지코드,총세대수,임대건물구분,지역_x,공급유형_x,전용면적,전용면적별세대수,공가수,자격유형,임대보증금_x,...,주소,세대수,공급유형_y,형 명,공급공용면적,주택유형,주차수,임대보증금_y,월임대료,준공일자
0,C2033,1400,아파트,울산광역시,영구임대,26.37,573,12.0,C,4032000,...,울산광역시 동구 월봉10길 16,984,영구임대,26,13.5000,아파트,255,3922000,72830,19910620
1,C2033,1400,아파트,울산광역시,영구임대,26.37,358,12.0,C,4032000,...,울산광역시 동구 월봉10길 16,984,영구임대,26,13.5000,아파트,255,3922000,72830,19910620
2,C2033,1400,아파트,울산광역시,영구임대,26.37,447,12.0,C,4032000,...,울산광역시 동구 월봉10길 16,984,영구임대,26,13.5000,아파트,255,3922000,72830,19910620
3,C1812,1021,아파트,울산광역시,영구임대,26.37,745,22.0,C,3922000,...,울산광역시 동구 월봉10길 16,984,영구임대,26,13.5000,아파트,255,3922000,72830,19910620
4,C1604,534,아파트,울산광역시,국민임대,51.49,174,13.0,A,16016000,...,울산광역시 북구 명촌로 94,534,국민임대,51,18.7651,아파트,400,16016000,208870,20030412
5,C1604,534,아파트,울산광역시,국민임대,51.95,360,13.0,A,16160000,...,울산광역시 북구 명촌로 94,534,국민임대,51,18.9327,아파트,400,16160000,210910,20030412
6,C2535,558,아파트,울산광역시,국민임대,51.86,462,23.0,A,21883000,...,울산광역시 북구 동대12길 30,558,국민임대,51,16.9528,아파트,480,21883000,202160,20040710
7,C2535,558,아파트,울산광역시,국민임대,59.99,96,23.0,A,25617000,...,울산광역시 북구 동대12길 30,558,국민임대,59,19.6104,아파트,480,25617000,254570,20040710
8,C1040,638,아파트,울산광역시,국민임대,36.63,345,24.0,A,14485000,...,울산광역시 북구 동대11길 36,638,국민임대,36,15.8602,아파트,548,14485000,126050,20040730
9,C1040,638,아파트,울산광역시,국민임대,46.19,293,24.0,A,18090000,...,울산광역시 북구 동대11길 36,638,국민임대,46,19.9996,아파트,548,18090000,184750,20040730


,단지코드,총세대수,임대건물구분,지역_x,공급유형_x,전용면적,전용면적별세대수,공가수,자격유형,임대보증금_x,...,주소,세대수,공급유형_y,형 명,공급공용면적,주택유형,주차수,임대보증금_y,월임대료,준공일자
0,C1377,470,아파트,경상남도,국민임대,36.63,234,18.0,A,9592000,...,경상남도 창녕군 창녕읍 말흘5길 33,338,국민임대,36,15.8760,아파트,239,9592000,70740,20070902.0
1,C2175,595,아파트,경상남도,공공임대(10년),84.94,13,5.0,A,58000000,...,경상남도 진주시 에나로228번길 10,404,10년임대,84,26.2327,아파트,0,47288000,627080,20171231.0
2,C2175,595,아파트,경상남도,공공임대(10년),84.94,11,5.0,A,58000000,...,경상남도 진주시 에나로228번길 10,404,10년임대,84,26.2327,아파트,0,47288000,627080,20171231.0
3,C2043,469,아파트,경상남도,행복주택,36.78,176,23.0,J,37240000,...,경상남도 진주시 대밭골로 91,966,행복주택,36,24.2656,아파트,879,34140000,167000,20180430.0
